[Kaggle March Madness Dataset](https://www.kaggle.com/c/mens-machine-learning-competition-2019/data)

In [148]:
# library for working with dataframes (matrics)
import pandas as pd

# Ensure Kaggle Stage2DataFiles are loaded in the following location: datasets/kaggle/stage_2

In [149]:
ls datasets/kaggle/stage_2

Cities.csv*                         RegularSeasonCompactResults.csv*
ConferenceTourneyGames.csv*         RegularSeasonDetailedResults.csv*
Conferences.csv*                    Seasons.csv*
GameCities.csv*                     SecondaryTourneyCompactResults.csv*
NCAATourneyCompactResults.csv*      SecondaryTourneyTeams.csv*
NCAATourneyDetailedResults.csv*     TeamCoaches.csv*
NCAATourneySeedRoundSlots.csv*      TeamConferences.csv*
NCAATourneySeeds.csv*               TeamSpellings.csv*
NCAATourneySlots.csv*               Teams.csv*


In [150]:
# Define file paths
FILE_PATH = 'datasets/kaggle/stage_2/'
NCAATourneyCompactResults_path = FILE_PATH + 'NCAATourneyCompactResults.csv'
NCAATourneySeeds_path = FILE_PATH + 'NCAATourneySeeds.csv'
RegularSeasonCompactResults_path = FILE_PATH + 'RegularSeasonCompactResults.csv'
Teams_path = FILE_PATH + 'Teams.csv'

In [151]:
# Create dataframes from files
NCAATourneyCompactResults = pd.read_csv(NCAATourneyCompactResults_path)
NCAATourneySeeds = pd.read_csv(NCAATourneySeeds_path)
RegularSeasonCompactResults = pd.read_csv(RegularSeasonCompactResults_path)
Teams = pd.read_csv(Teams_path)

In [152]:
# Look at dataframes
NCAATourneyCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [153]:
NCAATourneySeeds.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [154]:
RegularSeasonCompactResults.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


Teams.head()

## Create a 'seed' DataFrame

In [155]:
# Define a function that combines the values of two DataFrame columns,
# combining them into a single column with the values separated by an underscore
def get_id(p_row, p_column_names):
    column_values = [p_row[col] for col in p_column_names]
    return '_'.join(map(str, column_values))


# Make a copy of the NCAATourneySeeds DataFrame
seed = NCAATourneySeeds.copy()
# Add an 'id' column to the 'seed' DataFrame,
# using the get_id function to combine 'Season' and 'TeamID'
seed['id'] = seed.apply(lambda row: get_id(row, ['Season', 'TeamID']), axis=1)
# Extract the seed number from the 'Seed' value
seed['seed'] = seed['Seed'].apply(lambda x: int(x[1:3]))
# Extract the region from the 'Seed' value
seed['region'] = seed['Seed'].apply(lambda x: x[0])
# Rename the 'Season' column to 'season' and 'TeamID' to 'team'
seed = seed.rename(columns={'Season': 'season', 'TeamID': 'team'})
# Redefine the 'seed' DataFrame with the columns in this order
seed = seed[['id', 'season', 'team', 'region', 'seed']]
# Sort the 'seed' DataFrame by the 'id' column we created
seed = seed.sort_values(by=['id'])
seed.head()

,id,season,team,region,seed
22,1985_1104,1985,1104,X,7
25,1985_1112,1985,1112,X,10
24,1985_1116,1985,1116,X,9
58,1985_1120,1985,1120,Z,11
42,1985_1130,1985,1130,Y,11


In [156]:
# Look at NCAATourneyCompactResults
NCAATourneyCompactResults.head()
# Note 

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


## Create a 'final_four' DataFrame

In [157]:
# Create a 'tourney' DataFrame based on NCAATourneyCompactResults
tourney = NCAATourneyCompactResults.copy()
# Create a 'winners' DataFrame
winners = tourney[['Season', 'DayNum', 'WTeamID', 'WScore', 'NumOT']].copy()
# Rename the columns
winners = winners.rename(columns={'WTeamID': 'TeamID', \
                                  'WScore': 'Score'
                                 })
# Create a 'losers' DataFrame
losers = tourney[['Season', 'DayNum', 'LTeamID', 'LScore', 'NumOT']].copy()
# Rename the columns
losers = losers.rename(columns={'LTeamID': 'TeamID', \
                                'LScore': 'Score'
                               })
# Combine the 'winners' and 'losers' DataFrames
results = pd.concat([winners, losers])
# Create a 'final_four' DataFrame for only the teams that played on day 152
final_four = results.loc[results['DayNum'] == 152].copy()
# Create an 'id' column
final_four['id'] = final_four.apply(lambda row: get_id(row, ['Season', 'TeamID']), axis=1)
# Sort the DataFrame by the 'id' column
final_four = final_four[['id']].sort_values(by=['id'])
# Create a 'final_four' column, set it to 1
final_four['final_four'] = str(1) # string
# final_four['final_four'] = 1 # numeric
# Show the first 5 rows
final_four.head()

,id,final_four
60,1985_1207,1
61,1985_1272,1
60,1985_1385,1
61,1985_1437,1
123,1986_1181,1


## Create 'win_percentage' DataFrame

In [158]:
season = RegularSeasonCompactResults

# Create a 'winners' DataFrame
winners = season[['Season', 'DayNum', 'WTeamID', 'WScore', 'NumOT']].copy()
# Rename the columns
winners = winners.rename(columns={'WTeamID': 'TeamID', \
                                  'WScore': 'Score'
                                 })
winners['outcome'] = 'win'
# Create a 'losers' DataFrame
losers = season[['Season', 'DayNum', 'LTeamID', 'LScore', 'NumOT']].copy()
# Rename the columns
losers = losers.rename(columns={'LTeamID': 'TeamID', \
                                'LScore': 'Score'
                               })
losers['outcome'] = 'loss'
# Combine the 'winners' and 'losers' DataFrames
results = pd.concat([winners, losers])

results['id'] = results.apply(lambda row: get_id(row, ['Season', 'TeamID']), axis=1)


# Count win/loss by team by season
outcome_count = \
results[['id', 'outcome', 'TeamID']] \
.groupby(['id', 'outcome'], as_index=False) \
.count()

# Create win_count DataFrame
win_count = \
outcome_count\
.loc[outcome_count['outcome'] == 'win'] \
.rename(columns={'TeamID': 'win_count'})
print(win_count.head())
# Create loss_count DataFrame
loss_count = \
outcome_count \
.loc[outcome_count['outcome'] == 'loss'] \
.rename(columns={'TeamID': 'loss_count'})
print(loss_count.head())

          id outcome  win_count
1  1985_1102     win          5
3  1985_1103     win          9
5  1985_1104     win         21
7  1985_1106     win         10
9  1985_1108     win         19
          id outcome  loss_count
0  1985_1102    loss          19
2  1985_1103    loss          14
4  1985_1104    loss           9
6  1985_1106    loss          14
8  1985_1108    loss           6


In [159]:
outcome_count = \
results[['id', 'outcome', 'TeamID']] \
.groupby(['id', 'outcome'], as_index=False) \
.count()

win_count = \
outcome_count\
.loc[outcome_count['outcome'] == 'win'] \
.rename(columns={'TeamID': 'win_count'})
# print(win_count.head())
loss_count = \
outcome_count \
.loc[outcome_count['outcome'] == 'loss'] \
.rename(columns={'TeamID': 'loss_count'})
# print(loss_count.head())

pd.merge(win_count, loss_count, how='inner', on='id').head()

,id,outcome_x,win_count,outcome_y,loss_count
0,1985_1102,win,5,loss,19
1,1985_1103,win,9,loss,14
2,1985_1104,win,21,loss,9
3,1985_1106,win,10,loss,14
4,1985_1108,win,19,loss,6


In [160]:
win_percent = pd \
.merge(win_count, loss_count, how='inner', on='id')
win_percent['win_percent'] = win_percent['win_count'] / (win_percent['win_count'] + win_percent['loss_count'])
win_percent.head()

win_percent = win_percent[['id', 'win_percent']]
win_percent['win_percent'] = win_percent['win_percent'].apply(lambda x: '%.5f'%x)
win_percent['win_percent'] = win_percent['win_percent'].fillna(value=0)
win_percent.head()

,id,win_percent
0,1985_1102,0.20833
1,1985_1103,0.39130
2,1985_1104,0.70000
3,1985_1106,0.41667
4,1985_1108,0.76000


## Combine 'seed', 'win_percent', and 'final_four' DataFrames

In [161]:
# Merge the 'seed' and 'final_four' DataFrames.
# Left: Keep all the rows from the 'seed' DataFrame,
# even when there's no match in 'final_four'
dataset = seed
dataset = pd.merge(dataset, win_percent, how='left', on='id')
dataset = pd.merge(dataset, final_four, how='left', on='id')

dataset['win_percent'] = \
dataset['win_percent'] \
.convert_objects(convert_numeric=True) \
.fillna(0)

dataset.head()

/Users/ehelander/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  


,id,season,team,region,seed,win_percent,final_four
0,1985_1104,1985,1104,X,7,0.70000,NaN
1,1985_1112,1985,1112,X,10,0.66667,NaN
2,1985_1116,1985,1116,X,9,0.63636,NaN
3,1985_1120,1985,1120,Z,11,0.62069,NaN
4,1985_1130,1985,1130,Y,11,0.61538,NaN


In [162]:
# Set all the 'final_four' column values that aren't 1, to 0
dataset['final_four'] = dataset['final_four'].fillna(value=str(0)) # string
# Show the dataset the way we want it
dataset.head()

,id,season,team,region,seed,win_percent,final_four
0,1985_1104,1985,1104,X,7,0.70000,0
1,1985_1112,1985,1112,X,10,0.66667,0
2,1985_1116,1985,1116,X,9,0.63636,0
3,1985_1120,1985,1120,Z,11,0.62069,0
4,1985_1130,1985,1130,Y,11,0.61538,0


## Logistic regression

In [163]:
# Create a historical DataFrame (exclude 2019)
dataset_historical = dataset.loc[dataset['season'] != 2019]
dataset_historical.head()

,id,season,team,region,seed,win_percent,final_four
0,1985_1104,1985,1104,X,7,0.70000,0
1,1985_1112,1985,1112,X,10,0.66667,0
2,1985_1116,1985,1116,X,9,0.63636,0
3,1985_1120,1985,1120,Z,11,0.62069,0
4,1985_1130,1985,1130,Y,11,0.61538,0


In [164]:
# Create a 2019 DataFrame
dataset_2019 = dataset.loc[dataset['season'] == 2019]
dataset_2019.head()

,id,season,team,region,seed,win_percent,final_four
2218,2019_1101,2019,1101,Y,15,0.79310,0
2219,2019_1113,2019,1113,X,11,0.68750,0
2220,2019_1120,2019,1120,Y,5,0.73529,0
2221,2019_1124,2019,1124,X,9,0.59375,0
2222,2019_1125,2019,1125,W,11,0.83333,0


In [165]:
# We'll use 'X' for our predictors and 'y' for our target
# We just want the values, and we want it as an array instead of a DataFrame
X = dataset_historical.iloc[:, 4:6].values
X

array([[ 7.     ,  0.7    ],
       [10.     ,  0.66667],
       [ 9.     ,  0.63636],
       ...,
       [ 4.     ,  0.78125],
       [14.     ,  0.71875],
       [ 1.     ,  0.84848]])

In [166]:
y = dataset_historical.iloc[:, 6].values
y

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [167]:
# Split X and y into training and testing datasets

# # Via train_test_split
# from sklearn.model_selection import train_test_split
# X_train, X_test, \
# y_train, y_test = \
# train_test_split(X, \
#                  y, \
#                  test_size=0.3, \
#                  random_state=0 \
#                 )

# Via permutation
import numpy as np
# length of the dataset
length = dataset_historical.shape[0]
# We only want 70% of the data for the training set
train_index_stop = int(length * 0.7)
# Create a permutation
permutation = np.random.RandomState(0).permutation(length)
# Take just the values up to the train_index_stop
train_permutation = permutation[:train_index_stop]
# Take the values after the train_index_stop
test_permutation = permutation[train_index_stop:]

# Create the training and testing data
X_train = X[train_permutation]
X_test = X[test_permutation]
y_train = y[train_permutation]
y_test = y[test_permutation]

In [168]:
X_train

array([[ 9.     ,  0.66667],
       [ 7.     ,  0.86207],
       [10.     ,  0.63636],
       ...,
       [ 6.     ,  0.75   ],
       [ 6.     ,  0.68   ],
       [15.     ,  0.74074]])

In [169]:
y_train

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [170]:
# We want the predictors to be on the same scale
# (mean of 0, standard deviation of 1)
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
print('before')
print(X_train)

X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
# Don't worry about the warning
print('after')
print(X_train)

before
[[ 9.       0.66667]
 [ 7.       0.86207]
 [10.       0.63636]
 ...
 [ 6.       0.75   ]
 [ 6.       0.68   ]
 [15.       0.74074]]
after
[[ 0.07074208 -0.54278177]
 [-0.35729575  1.32715855]
 [ 0.28476099 -0.83284262]
 ...
 [-0.57131467  0.25467027]
 [-0.57131467 -0.41521624]
 [ 1.35485557  0.16605385]]


In [171]:
# Fit a LogisticRegression object to training set
from sklearn.linear_model import LogisticRegression
classifierObj = LogisticRegression(random_state=0)
classifierObj.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [172]:
y_train

array(['0', '0', '0', ..., '0', '0', '0'], dtype=object)

In [173]:
# Making predictions on the test set
y_pred = classifierObj.predict(X_test)

In [174]:
# Evaluating the predictions using a confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

# So, we predicted 0 Trues correctly.
# This is to be expected, because the model, by default,
# will only predict a class if it has a greater than
# 50% probability. None of these teams do.

[[628   0]
 [ 38   0]]


In [175]:
# Predict the probability of [0, 1]
prob = classifierObj.predict_proba(X_test)
prob

array([[0.99561857, 0.00438143],
       [0.98992364, 0.01007636],
       [0.83578787, 0.16421213],
       ...,
       [0.83207297, 0.16792703],
       [0.97698283, 0.02301717],
       [0.99877778, 0.00122222]])

In [176]:
test_predicted_probability = prob[:, 1]
print(test_predicted_probability.shape)
# Predicted y probabilities
test_predicted_probability

(666,)


array([0.00438143, 0.01007636, 0.16421213, 0.00078303, 0.17005804,
       0.16792703, 0.1153579 , 0.1664995 , 0.03438705, 0.00079115,
       0.16676429, 0.3214357 , 0.00187335, 0.23461091, 0.02244938,
       0.0007768 , 0.01571569, 0.03451167, 0.32466428, 0.05170948,
       0.16738344, 0.00283088, 0.17264122, 0.00076861, 0.01032369,
       0.23784377, 0.02416545, 0.00995918, 0.00654923, 0.03480411,
       0.00121311, 0.00430694, 0.0232977 , 0.00078096, 0.00289768,
       0.03480411, 0.00186452, 0.00665645, 0.02291437, 0.00450653,
       0.23406216, 0.11620208, 0.051938  , 0.23732208, 0.11732875,
       0.1675361 , 0.23612778, 0.01538211, 0.01533678, 0.16676429,
       0.32605942, 0.32227952, 0.07866566, 0.05415127, 0.00124996,
       0.03459116, 0.00672675, 0.0232757 , 0.1688747 , 0.23600484,
       0.02360268, 0.03456994, 0.32188287, 0.00078156, 0.00124996,
       0.16871636, 0.05228626, 0.03487994, 0.01515682, 0.02312781,
       0.32092713, 0.00439488, 0.16738344, 0.00429299, 0.07892

In [177]:
# Actual y values
y_test

array(['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '1', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1',
       '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '1', '0', '1', '0', '0', '0', '0',
       '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0',
       '0', '0', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0

In [178]:
# Get, show original data for test split
dataset_historical_test = dataset_historical.loc[test_permutation]
dataset_historical_test.head()

,id,season,team,region,seed,win_percent,final_four
1746,2012_1143,2012,1143,X,12,0.72727,0
753,1996_1400,1996,1400,X,10,0.68966,0
462,1992_1199,1992,1199,X,3,0.67857,0
673,1995_1291,1995,1291,Z,16,0.57143,0
1682,2011_1140,2011,1140,Y,3,0.87879,0


In [179]:
# Add predicted probability to historical data
dataset_historical_test['predicted_probability'] = test_predicted_probability
dataset_historical_test

,id,season,team,region,seed,win_percent,final_four,predicted_probability
1746,2012_1143,2012,1143,X,12,0.72727,0,0.004381
753,1996_1400,1996,1400,X,10,0.68966,0,0.010076
462,1992_1199,1992,1199,X,3,0.67857,0,0.164212
673,1995_1291,1995,1291,Z,16,0.57143,0,0.000783
1682,2011_1140,2011,1140,Y,3,0.87879,0,0.170058
1463,2007_1401,2007,1401,X,3,0.80645,0,0.167927
1201,2003_1390,2003,1390,X,4,0.74194,0,0.115358
2203,2018_1397,2018,1397,Y,3,0.75758,0,0.166500
424,1991_1345,1991,1345,W,7,0.60714,0,0.034387
325,1990_1131,1990,1131,W,16,0.62069,0,0.000791


In [180]:
# Scale 2019 data
X_2019 = dataset_2019.iloc[:, 4:6].values
X_2019 = sc_X.transform(X_2019)

# 2019 output
dataset_2019_output = dataset_2019.copy()
dataset_2019_output = dataset_2019_output[['id', 'team', 'region', 'seed', 'win_percent']]
dataset_2019_output['predicted_probability'] = classifierObj.predict_proba(X_2019)[:, 1]
dataset_2019_output

,id,team,region,seed,win_percent,predicted_probability
2218,2019_1101,1101,Y,15,0.79310,0.001252
2219,2019_1113,1113,X,11,0.68750,0.006619
2220,2019_1120,1120,Y,5,0.73529,0.078583
2221,2019_1124,1124,X,9,0.59375,0.015004
2222,2019_1125,1125,W,11,0.83333,0.006824
2223,2019_1133,1133,W,15,0.57576,0.001196
2224,2019_1138,1138,X,6,0.90909,0.054758
2225,2019_1153,1153,Z,7,0.82353,0.035927
2226,2019_1159,1159,Z,15,0.69697,0.001227
2227,2019_1181,1181,W,1,0.85294,0.322087


## Prediction!

In [181]:
idx = dataset_2019_output.groupby(['region'])['predicted_probability'].transform(max) == dataset_2019_output['predicted_probability']
dataset_2019_output[idx]

,id,team,region,seed,win_percent,predicted_probability
2227,2019_1181,1181,W,1,0.85294,0.322087
2233,2019_1211,1211,X,1,0.90909,0.324664
2258,2019_1314,1314,Y,1,0.81818,0.320497
2280,2019_1438,1438,Z,1,0.90625,0.324534


In [182]:
predicted_4 = dataset_2019_output[idx]['team']
predicted_4 = pd.DataFrame(predicted_4).rename(columns={'team': 'TeamID'})

pd.merge(predicted_4, Teams, on='TeamID', how='inner')

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1181,Duke,1985,2019
1,1211,Gonzaga,1985,2019
2,1314,North Carolina,1985,2019
3,1438,Virginia,1985,2019
